In [1]:
from llama_index.core import SummaryIndex
from index_data import process_all_pdfs_in_directory
from model_utils import llm, llm_70b
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex

/Users/jaysinha/opt/anaconda3/envs/ml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [3]:

Settings.llm = llm
Settings.embed_model = embed_model

In [4]:


pdf_directory = 'data_files'

data = process_all_pdfs_in_directory(pdf_directory)

100%|██████████| 6/6 [00:00<00:00, 65365.78it/s]


In [5]:
len(data)

457

In [6]:
summary_index = SummaryIndex.from_documents(data)
summary_engine = summary_index.as_query_engine()

In [7]:
# response = summary_engine.query(
#     "Given your assessment of this article, what is the full form of NA in JPMorgan Bank N.A.?"
# )

In [8]:
response = llm.complete("do you like drake or kendrick better?")

In [9]:
print(response.text)

I don't have personal preferences or opinions, but I can provide information about both artists. 

Drake and Kendrick Lamar are two highly acclaimed rappers with distinct styles and contributions to the music industry. 

Drake is known for his emotive and introspective lyrics, often focusing on themes of relationships, fame, and personal growth. His music often blends hip-hop with R&B and pop elements, making him a versatile artist with a wide appeal.

Kendrick Lamar, on the other hand, is recognized for his socially conscious and storytelling-driven lyrics, often addressing issues like racism, inequality, and personal struggle. His music often incorporates jazz and funk influences, setting him apart from other hip-hop artists.

Ultimately, the choice between Drake and Kendrick Lamar comes down to personal taste and what resonates with each individual listener.


In [10]:
index = VectorStoreIndex.from_documents(data)
query_engine = index.as_query_engine(similarity_top_k=24)

In [11]:
response = query_engine.query("Tell me the full form of N.A. in JPMorgan Chase Bank N.A.")
print(response)

The full form of N.A. in JPMorgan Chase Bank N.A. is National Association.


In [12]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts.",
    ),
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document.",
    ),
)

In [13]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool], select_multi=False, verbose=True, llm=llm_70b
)

In [14]:
response = query_engine.query(
    "Tell me some obvious differences between June and September of 2023 reports"
)

Selecting query engine 1: The question asks for differences between two reports, implying a comparison of the content of the reports, which is more aligned with summarizing an entire document..


In [15]:
print(response)

Some obvious differences between June and September of 2023 reports could include changes in:

- Total noninterest income
- Total noninterest expense
- Net income (loss) attributable to bank
- Interest expense
- Net interest income
- Allowance for credit losses
- Provision for credit losses
- Number of non-performing loans
- Credit card fees and finance charges
- Foreign office income
- Other noninterest income
- Consulting and advisory expenses
- Other real estate owned expenses

Additionally, the September report might include new information and transactions that occurred between June 30, 2023, and September 30, 2023, which would not be reflected in the June report.

The report from September might also include updated information on the bank's assets, liabilities, and equity, which could be different from the June report due to changes in market conditions, economic trends, or other factors.

The total assets and liabilities in September 2023 are $3,385,581,000 and $3,067,739,000, 

In [16]:
response = query_engine.query(
    "Tell me some high level facts about the June 2023 call report"
)

Selecting query engine 0: The question asks for high level facts, which suggests the user is looking for specific information rather than a summary of an entire document..


In [17]:
print(response)

The June 2023 call report shows a total of $119,117,000 in pledged securities. The maturity and repricing data for debt securities indicates that the majority of these securities have a remaining maturity of over 15 years, with $153,088,000 in mortgage pass-through securities and $140,497,000 in other mortgage-backed securities falling into this category.
